<a href="https://colab.research.google.com/github/VaibhavvvBarot/JsonOptimizer/blob/main/JsonCleaner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### *JsonOptimizer*

*   Assign
*   Conditional
*   Query
*   Comment
*   Hide/Show/Enable/Disable
*   Call Logic Block
*   Update/Delete Record
*   Create Message
*   Interrupt with error
*   Add
*   Subtract
*   Divide
*   Multiply
*   Query Subtable

In [1]:
import json
from google.colab import output
from IPython.display import display, HTML

In [88]:
# Define the valid keys for each type
LOGIC_CONTROL_KEYS = [
    "ValueType",
    "DocSourceName",
    "Value",
    "DataType",
    "ValueSubType",
    "DataSubType",
    "LookupType",
    "LookupName"
]

LOGIC_EXPRESSION_ADDITIONAL_KEYS = [
    "ConditionalOperator",
    "ID",
    "ExpressionType",
    "CompareToValue",
    "CompareToValueType",
    "CompareToDocSourceName",
    "CompareToDataType",
    "CompareToDataSubType",
    "CompareToLookupType",
    "LogicalOperator"
]

# Action-specific keys
ASSIGN_VALUE_OUTPUT_KEYS = ["AssignAction"]
HIDE_SHOW_ENABLE_KEYS = ["Index"]  # Used for other actions, not for UI field actions
UI_FIELD_ADDITIONAL_KEYS = [
    "UiHintType",
    "DetailApplicationName",
    "SubField"
]
QUERY_ADDITIONAL_KEYS = ["QueryType", "SortType"]
CALL_LOGIC_BLOCK_KEYS = [
    "logicBlockMode",
    "logicBlockName",
    "adoptAllMessages",
    "LbRequireNewTxn",
    "errorList"
]

# Insert action specific keys
INSERT_ACTION_KEYS = [
    "logicSchema",
    "DocSourceName",
    "cdaMappings",
    "HDStructureName",
    "HeaderDocSourceName",
    "DetailDocSourceName",
    "HDHeaderTableSchema",
    "sourceDocSourceName",
    "TableSchemaType",
    "HDStructureHeaderOrDetail"
]

# Update/Delete action specific keys
UPDATE_DELETE_KEYS = [
    "logicSchema",
    "DocSourceName"
]

# Create Message action specific keys
CREATE_MESSAGE_KEYS = [
    "messageSourceType",
    "dataItem",
    "messageType",
    "messageSeverity",
    "inputMessage",
    "ErrorSource"
]

# Interrupt With Error action specific keys
INTERRUPT_ERROR_KEYS = [
    "dataItem",
    "messageType",
    "ErrorSource"
]

# Insert input item specific keys
INSERT_INPUT_KEYS = [
    "fieldName",
    "fieldType",
    "dataType",
    "docSourceNameFrom",
    "fieldLookupType",
    "LookupType",
    "LookupName",
    "value",
    "variable",
    "dataSubType",
    "valueSubType",
    "bulkOperation",
    "mappingName",
    "mappingShape",
    "dataItemDefinition",
    "required"
]

# Keys to always include for any logic action
COMMON_LOGIC_ACTION_KEYS = ["logicAction"]

# Query action specific top-level keys
QUERY_TOP_LEVEL_KEYS = [
    "DocSourceName",
    "logicSchema",
    "logicExpression",
    "logicQuery",
    "logicInstanceFalse",
    "logicInstanceTrue",
    "Dispositions"
]

# List of UI field action types
UI_FIELD_ACTIONS = ["Show Fields", "Hide Fields", "Enable Fields", "Disable Fields"]

# Add math action specific keys
MATH_ACTION_OUTPUT_COMMON_KEYS = ["Round"]
MINUS_ACTION_ADDITIONAL_KEYS = ["DateUnit"]
MATH_ACTIONS = ["Sum", "Minus", "Divide", "Multiply"]

# Query Subtable specific keys
QUERY_SUBTABLE_KEYS = [
    "DocSourceName",
    "selectCurrentRow",
    "parentDocSourceName",
    "parentFieldName",
    "logicSchema",
    "logicInstanceTrue",
    "logicInstanceFalse"
]

def filter_logic_control(item):
    """Filter item to only include valid LogicControl keys that are present"""
    filtered_item = {}
    for key in LOGIC_CONTROL_KEYS:
        if key in item:
            filtered_item[key] = item[key]
    return filtered_item

def filter_logic_expression(item):
    """Filter item to include both LogicControl and LogicExpression keys that are present"""
    filtered_item = filter_logic_control(item)
    for key in LOGIC_EXPRESSION_ADDITIONAL_KEYS:
        if key in item:
            filtered_item[key] = item[key]
    return filtered_item

def filter_query_item(item):
    """Filter query items to include LogicControl and Query specific keys"""
    filtered_item = filter_logic_control(item)
    for key in QUERY_ADDITIONAL_KEYS:
        if key in item:
            filtered_item[key] = item[key]
    return filtered_item

def filter_ui_field_item(item):
    """Filter UI field items to include LogicControl and UI field specific keys"""
    filtered_item = filter_logic_control(item)
    for key in UI_FIELD_ADDITIONAL_KEYS:
        if key in item:
            filtered_item[key] = item[key]
    return filtered_item

def filter_insert_input_item(item):
    """Filter insert input items to include specific keys"""
    filtered_item = {}
    for key in INSERT_INPUT_KEYS:
        if key in item:
            filtered_item[key] = item[key]

    # Handle nested input specially
    if "NestedInput" in item and item["NestedInput"]:
        filtered_item["NestedInput"] = filter_logic_control(item["NestedInput"])

    return filtered_item

def optimize_logic_action(input_json):
    """Recursively optimize a logic action of any type"""
    if not isinstance(input_json, dict):
        return input_json

    # Keep track of the action type
    logic_action = input_json.get("logicAction", "")

    # Initialize the result with the required keys
    result = {"logicAction": logic_action}

    # Process based on the specific logicAction type
    if logic_action == "Assign Value":
        # Process logicInstanceInputs
        if "logicInstanceInputs" in input_json:
            result["logicInstanceInputs"] = [
                filter_logic_control(item) for item in input_json["logicInstanceInputs"]
            ]

        # Process logicInstanceOutputs
        if "logicInstanceOutputs" in input_json:
            outputs = []
            for item in input_json["logicInstanceOutputs"]:
                filtered_item = filter_logic_control(item)
                # Add action-specific keys
                for key in ASSIGN_VALUE_OUTPUT_KEYS:
                    if key in item:
                        filtered_item[key] = item[key]
                outputs.append(filtered_item)
            result["logicInstanceOutputs"] = outputs

    elif logic_action == "Conditional":
        # Process logicExpression
        if "logicExpression" in input_json:
            result["logicExpression"] = [
                filter_logic_expression(item) for item in input_json["logicExpression"]
            ]

        # Process nested logicInstanceTrue - maintain the structure with "logicInstance"
        if "logicInstanceTrue" in input_json:
            true_instances = []
            for item in input_json["logicInstanceTrue"]:
                if "logicInstance" in item:
                    # Recursively optimize the nested logicInstance
                    optimized_instance = optimize_logic_action(item["logicInstance"])
                    true_instances.append({"logicInstance": optimized_instance})
                else:
                    # If it doesn't have a logicInstance, keep it as is
                    true_instances.append(item)
            result["logicInstanceTrue"] = true_instances

        # Process nested logicInstanceFalse - maintain the structure with "logicInstance"
        if "logicInstanceFalse" in input_json:
            false_instances = []
            for item in input_json["logicInstanceFalse"]:
                if "logicInstance" in item:
                    # Recursively optimize the nested logicInstance
                    optimized_instance = optimize_logic_action(item["logicInstance"])
                    false_instances.append({"logicInstance": optimized_instance})
                else:
                    # If it doesn't have a logicInstance, keep it as is
                    false_instances.append(item)
            result["logicInstanceFalse"] = false_instances

    # Handle UI field actions (Show Fields, Hide Fields, Enable Fields, Disable Fields)
    elif logic_action in UI_FIELD_ACTIONS:
        if "logicInstanceInputs" in input_json:
            inputs = []
            for item in input_json["logicInstanceInputs"]:
                # Use the specialized UI field filter function (without Index)
                filtered_item = filter_ui_field_item(item)
                inputs.append(filtered_item)
            result["logicInstanceInputs"] = inputs

        # These actions typically don't have outputs, but handle it just in case
        if "logicInstanceOutputs" in input_json and input_json["logicInstanceOutputs"]:
            result["logicInstanceOutputs"] = input_json["logicInstanceOutputs"]

    # Handle Query action
    elif logic_action == "Query":
        # Add top-level keys for Query action
        for key in QUERY_TOP_LEVEL_KEYS:
            if key in input_json:
                if key == "logicExpression":
                    # Process logicExpression array
                    result[key] = [filter_logic_expression(item) for item in input_json[key]]
                elif key == "logicQuery":
                    # Process logicQuery array with additional query-specific keys
                    result[key] = [filter_query_item(item) for item in input_json[key]]
                elif key == "Dispositions" and isinstance(input_json[key], dict):
                    # Copy Dispositions object as is
                    result[key] = {k: v for k, v in input_json[key].items()
                                if k in ["Created", "Changed", "Unchanged", "Removed"]}
                elif key in ["logicInstanceTrue", "logicInstanceFalse"]:
                    # Process nested actions
                    instances = []
                    for item in input_json[key]:
                        if "logicInstance" in item:
                            optimized_instance = optimize_logic_action(item["logicInstance"])
                            instances.append({"logicInstance": optimized_instance})
                        else:
                            instances.append(item)
                    result[key] = instances
                else:
                    # Copy other top-level keys directly
                    result[key] = input_json[key]

    # Handle Call Logic Block V2 action
    elif logic_action == "Call Logic Block V2":
        # Add the specific Call Logic Block keys
        for key in CALL_LOGIC_BLOCK_KEYS:
            if key in input_json:
                result[key] = input_json[key]

        # Note: We're purposefully leaving out inputs and outputs handling for Call Logic Block V2
        # as specified in the requirements

    # Handle Insert action
    elif logic_action == "Insert":
        # Add top-level keys for Insert action
        for key in INSERT_ACTION_KEYS:
            if key in input_json:
                result[key] = input_json[key]

        # Process logicInsertInputs if present
        if "logicInsertInputs" in input_json:
            result["logicInsertInputs"] = [
                filter_insert_input_item(item) for item in input_json["logicInsertInputs"]
            ]

    # Handle Update and Delete actions
    elif logic_action in ["Update", "Delete"]:
        # Add the simple top-level keys for Update/Delete actions
        for key in UPDATE_DELETE_KEYS:
            if key in input_json:
                result[key] = input_json[key]

    # Handle Create Message action
    elif logic_action == "Create Message":
        # Add top-level keys for Create Message action
        for key in CREATE_MESSAGE_KEYS:
            if key in input_json:
                if key in ["dataItem", "messageType", "messageSeverity"]:
                    # These are objects with LogicControl-like keys
                    result[key] = filter_logic_control(input_json[key])
                elif key == "ErrorSource" and isinstance(input_json[key], list):
                    # This is an array of objects with LogicControl-like keys
                    result[key] = [filter_logic_control(item) for item in input_json[key]]
                else:
                    # Handle other top-level keys
                    result[key] = input_json[key]

        # Process logicInstanceInputs
        if "logicInstanceInputs" in input_json:
            result["logicInstanceInputs"] = [
                filter_logic_control(item) for item in input_json["logicInstanceInputs"]
            ]

    # Handle Interrupt With Error action
    elif logic_action == "Interrupt With Error":
        # Add top-level keys for Interrupt With Error action
        for key in INTERRUPT_ERROR_KEYS:
            if key in input_json:
                if key in ["dataItem", "messageType"]:
                    # These are objects with LogicControl-like keys
                    result[key] = filter_logic_control(input_json[key])
                elif key == "ErrorSource" and isinstance(input_json[key], list):
                    # This is an array of objects with LogicControl-like keys
                    result[key] = [filter_logic_control(item) for item in input_json[key]]

        # Process logicInstanceInputs
        if "logicInstanceInputs" in input_json:
            result["logicInstanceInputs"] = [
                filter_logic_control(item) for item in input_json["logicInstanceInputs"]
            ]

    # Handle math actions (Sum, Minus, Divide, Multiply)
    elif logic_action in MATH_ACTIONS:
        # Process logicInstanceInputs
        if "logicInstanceInputs" in input_json:
            result["logicInstanceInputs"] = [
                filter_logic_control(item) for item in input_json["logicInstanceInputs"]
            ]

        # Process logicInstanceOutputs with math-specific keys
        if "logicInstanceOutputs" in input_json:
            outputs = []
            for item in input_json["logicInstanceOutputs"]:
                filtered_item = filter_logic_control(item)

                # Add common math action keys
                for key in MATH_ACTION_OUTPUT_COMMON_KEYS:
                    if key in item:
                        filtered_item[key] = item[key]

                # Add Minus-specific keys if this is a Minus action
                if logic_action == "Minus":
                    for key in MINUS_ACTION_ADDITIONAL_KEYS:
                        if key in item:
                            filtered_item[key] = item[key]

                outputs.append(filtered_item)
            result["logicInstanceOutputs"] = outputs

    # Handle Query Subtable action
    elif logic_action == "Query Subtable":
        # Add the specific Query Subtable keys
        for key in QUERY_SUBTABLE_KEYS:
            if key in input_json and key not in ["logicInstanceTrue", "logicInstanceFalse"]:
                result[key] = input_json[key]

        # Process nested logicInstanceTrue - maintain the structure with "logicInstance"
        if "logicInstanceTrue" in input_json:
            true_instances = []
            for item in input_json["logicInstanceTrue"]:
                if "logicInstance" in item:
                    # Recursively optimize the nested logicInstance
                    optimized_instance = optimize_logic_action(item["logicInstance"])
                    true_instances.append({"logicInstance": optimized_instance})
                else:
                    # If it doesn't have a logicInstance, keep it as is
                    true_instances.append(item)
            result["logicInstanceTrue"] = true_instances

        # Process nested logicInstanceFalse - maintain the structure with "logicInstance"
        if "logicInstanceFalse" in input_json:
            false_instances = []
            for item in input_json["logicInstanceFalse"]:
                if "logicInstance" in item:
                    # Recursively optimize the nested logicInstance
                    optimized_instance = optimize_logic_action(item["logicInstance"])
                    false_instances.append({"logicInstance": optimized_instance})
                else:
                    # If it doesn't have a logicInstance, keep it as is
                    false_instances.append(item)
            result["logicInstanceFalse"] = false_instances

    # For any other logic action type, just keep the basic structure
    else:
        # Handle inputs if present
        if "logicInstanceInputs" in input_json:
            result["logicInstanceInputs"] = [
                filter_logic_control(item) for item in input_json["logicInstanceInputs"]
            ]

        # Handle outputs if present
        if "logicInstanceOutputs" in input_json:
            result["logicInstanceOutputs"] = [
                filter_logic_control(item) for item in input_json["logicInstanceOutputs"]
            ]

        # Handle nested true/false branches if present
        for branch in ["logicInstanceTrue", "logicInstanceFalse"]:
            if branch in input_json:
                branch_instances = []
                for item in input_json[branch]:
                    if "logicInstance" in item:
                        optimized_instance = optimize_logic_action(item["logicInstance"])
                        branch_instances.append({"logicInstance": optimized_instance})
                    else:
                        branch_instances.append(item)
                result[branch] = branch_instances

    return result

# Get JSON input from user
json_text = input("Paste your JSON here: ")

try:
    import json
    from IPython.display import HTML

    input_json = json.loads(json_text)
    optimized_json = optimize_logic_action(input_json)

    # Format the optimized JSON
    formatted_json = json.dumps(optimized_json, indent=2)

    # Clear previous output
    try:
        from IPython.display import clear_output
        clear_output()
    except:
        pass  # Not in an IPython environment

    # Create a text area with the optimized JSON for easy copying
    display(HTML(f"""
    <h3>Optimized JSON (Click inside the box, press Ctrl+A to select all, then Ctrl+C to copy):</h3>
    <textarea id="json-output" style="width:100%; height:500px; font-family:monospace; white-space:pre;">{formatted_json}</textarea>
    <script>
        // Auto-select the text for easy copying
        var textarea = document.getElementById('json-output');
        textarea.focus();
        textarea.select();
    </script>
    """))

    print("\nThe optimized JSON is ready to copy from the text area above.")

except Exception as e:
    print(f"Error: {str(e)}")


The optimized JSON is ready to copy from the text area above.
